In [ ]:
import numpy as np
import pandas as pd

from power_grid_model import LoadGenType, ComponentType, DatasetType, ComponentAttributeFilterOptions
from power_grid_model import PowerGridModel, CalculationMethod, CalculationType
from power_grid_model import initialize_array, attribute_dtype
from power_grid_model import FaultType, FaultPhase,ShortCircuitVoltageScaling

In [ ]:
def Y_calculation(line_data):
    """
    计算 导纳矩阵
    """
    # 初始化33x33的零矩阵
    Y = np.zeros((33, 33), dtype=complex)
    # 计算自导纳和互导纳
    for i in range(len(line)):
        from_node = int(line[i]['from_node'])
        to_node = int(line[i]['to_node'])
        r = line[i]['r1']
        x = line[i]['x1']
        
        # 计算自导纳
        y_ii = 1 / (r + 1j * x)
        
        # 添加自导纳
        Y[from_node, from_node] += y_ii
        Y[to_node, to_node] += y_ii
        
        # 添加互导纳
        Y[from_node, to_node] -= y_ii
        Y[to_node, from_node] -= y_ii
        
    return Y

In [ ]:
def I_calculation(Y_maxtri,V_maxtri):
    V = pd.DataFrame(V_maxtri)
    V_np = V.to_numpy()#转换为numpy
    I=np.matmul(Y_maxtri,V_np)#节点电流矩阵
    return I

In [ ]:
def transform_abs_angle(I_abi):
    # 计算模值
    magnitudes = np.abs(I_abi)
    
    # 计算角度（默认以弧度为单位）
    angles = np.angle(I_abi)
    
    # 将模值和角度组合成一个新的二维数组
    I_abs_angle = np.column_stack((magnitudes, angles))
    I_abs_angle = np.around(I_abs_angle,4)
    return I_abs_angle

In [ ]:
#节点
node = initialize_array(DatasetType.input, ComponentType.node, 33)
node['id'] = np.arange(33)#节点编号
node['u_rated'] = 10.5e3#额定电压

#线路
line = initialize_array(DatasetType.input, ComponentType.line, 40)
line['id'] = np.arange(33,73)#线路编号
df_bus = pd.read_csv('Bus.csv')#读取节点文件
from_node = df_bus.iloc[:, 0]#提取第一列
to_node = df_bus.iloc[:, 1]#提取第二列
#转换为numpy
from_node = from_node.to_numpy()
to_node = to_node.to_numpy()
#赋值
line['from_node'] = from_node#线路起始节点
line['to_node'] = to_node#线路终止节点
r_node = df_bus.iloc[:, 2]#提取第三列电阻
x_node = df_bus.iloc[:, 3]#提取第四列电抗
#转换为numpy
r_node = r_node.to_numpy()
x_node = x_node.to_numpy()
#赋值
line['r1'] = r_node
line['x1'] = x_node
line['c1'] = 0
line['tan1'] = 0
line['r0']= 0.05
line['x0'] = 0.05
line['c0'] = 0
line['tan0'] = 0
line['from_status'] = 1
line['to_status'] = 1
line['i_n'] = 400

#负载
load = initialize_array(DatasetType.input, ComponentType.sym_load, 33)
load['id'] = np.arange(73, 106)#节点编号
df_load = pd.read_csv('loads.csv')#读取文件
load_node = df_load.iloc[:, 0]#提取第一列
load_p = df_load.iloc[:, 1]#提取第二列
load_q = df_load.iloc[:, 2]#提取第三列
#转换为numpy
load_node = load_node.to_numpy()
load_p = load_p.to_numpy()*1e6
load_q = load_q.to_numpy()*1e6
load['node'] = load_node
load['type'] = LoadGenType.const_power
load['p_specified'] = load_p
load['q_specified'] = load_q
load['status'] = 1

#电源
source = initialize_array(DatasetType.input, ComponentType.source, 1)
source['id'] = 106
source['node'] = 0
source['status'] = 1
source['u_ref'] = 1.2057142857#电压参考值

#故障（单相接地）
fault = initialize_array(DatasetType.input, ComponentType.fault, 1)
fault["id"] = [107]
fault["status"] = [1]
fault["fault_object"] = [8]#在节点8处故障
fault["fault_type"] = [FaultType.single_phase_to_ground]
fault["fault_phase"] = [FaultPhase.a]
fault["r_f"] = [15]
fault["x_f"] = [0.1]
# all
input_data = {
    ComponentType.node: node,
    ComponentType.line: line,
    ComponentType.sym_load: load,
    ComponentType.source: source,
    ComponentType.fault: fault,
}



In [ ]:
from power_grid_model.validation import assert_valid_input_data

assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.short_circuit)

In [ ]:
model = PowerGridModel(input_data)

In [ ]:
sc_res = model.calculate_short_circuit()

In [ ]:

print("\n------fault node result: node------")
print(fault["fault_object"])
print("------fault result: i_f------")
print(sc_res[ComponentType.fault]["i_f"])
print("------fault result: i_to------")
print(sc_res[ComponentType.line]["i_to"])
print("\n------node result: id------")
print(sc_res[ComponentType.node]["id"])
print("------node result: u------")
print(sc_res[ComponentType.node]["u"])
print("------node result: u_angle------")
print(sc_res[ComponentType.node]["u_angle"])


In [ ]:
# 定义 CSV 文件名
csv_file = 'shortcircuit1.csv'
with open(csv_file, 'w') as f:
    f.write('fault_node,node,ua,ua_angle,ub,ub_angle,uc,uc_angle,ia,ia_angle,ib,ib_angle,ic,ic_angle\n')  # 写入表头


In [ ]:
fault_node = fault["fault_object"]
fault_node = [fault_node[0]] * 33
node = sc_res[ComponentType.node]["id"]
u_sc = sc_res[ComponentType.node]["u"]
u_sc = np.around(u_sc,4)
ua = [row[0] for row in u_sc]
ub = [row[1] for row in u_sc]
uc = [row[2] for row in u_sc]
u_angle = sc_res[ComponentType.node]["u_angle"]
u_angle = np.around(u_angle,4)
ua_angle = [row[0] for row in u_angle]
ub_angle = [row[1] for row in u_angle]
uc_angle = [row[2] for row in u_angle]


In [ ]:
#计算导纳矩阵,节点电流矩阵
Y = Y_calculation(input_data[ComponentType.line])
Ia = I_calculation(Y,ua)
Ib = I_calculation(Y,ub)
Ic = I_calculation(Y,uc)
Ia = transform_abs_angle(Ia)
Ib = transform_abs_angle(Ib)
Ic = transform_abs_angle(Ic)

In [ ]:

ia = [row[0] for row in Ia]
ib = [row[0] for row in Ib]
ic = [row[0] for row in Ic]
ia_angle = [row[1] for row in Ia]
ib_angle = [row[1] for row in Ib]
ic_angle = [row[1] for row in Ic]

sc_res2 = pd.DataFrame({
        'fault_node': fault_node,
        'node': node,
        'ua': ua,
        'ua_angle': ua_angle,
        'ub': ub,
        'ub_angle': ub_angle,
        'uc': uc,
        'uc_angle': uc_angle,
        'ia': ia,
        'ia_angle': ia_angle,
        'ib': ib,
        'ib_angle': ib_angle,
        'ic': ic,
        'ic_angle': ic_angle

        
    })
sc_res2.to_csv(csv_file, mode='a', header=False, index=False)